In [3]:
import praw
import pandas as pd
import time

# Set up Reddit API credentials
user_agent = "Scraper 1.0 by /u/Then_Basil_7613"
reddit = praw.Reddit(
    client_id="JZvwf0yO8TYu4TnCVVkhdQ",
    client_secret="V46c7n3SENTe5BCyCYDvnQsQQ-XlRA",
    user_agent=user_agent
)

# Create a list to store submission information
posts_data = []

# Track number of entries collected
count = 0
MAX_POSTS = 5000  # Adjust this as needed

# Define subreddit
subreddit = reddit.subreddit('AskReddit')

# Collect submissions in a backward manner
last_submission = None

while count < MAX_POSTS:
    try:
        # Define how many posts to collect in each iteration (chunk size)
        limit_per_iteration = 1000

        # Get new posts starting from the last post collected (if any)
        if last_submission:
            submissions = subreddit.new(limit=limit_per_iteration, params={"before": last_submission})
        else:
            submissions = subreddit.new(limit=limit_per_iteration)

        # Store posts data
        post_count_in_iteration = 0

        for submission in submissions:
            # Extract submission data
            post = {
                'User': submission.author.name if submission.author else 'Deleted',
                'Post Title': submission.title,
                'Content': submission.selftext,  # The content of the post (important for linguistic analysis)
                'Score': submission.score,  # Upvotes received, can indicate post engagement
                'Comments Count': submission.num_comments,  # Number of comments, useful for understanding engagement
                'Upvote Ratio': submission.upvote_ratio,  # Ratio of upvotes to total votes
                'Flair': submission.link_flair_text if submission.link_flair_text else 'None',  # Flair can help classify types of posts
                'Date': pd.to_datetime(submission.created_utc, unit='s'),
                'Post ID': submission.id  # To link comments to the post
            }

            posts_data.append(post)
            count += 1
            post_count_in_iteration += 1

            # Track the last submission id to use it for the next iteration
            last_submission = submission.name

            # Stop if we have collected enough posts
            if count >= MAX_POSTS:
                break

        # If no new posts were collected in the iteration, we might have reached the end of available data
        if post_count_in_iteration == 0:
            print("No more posts available to collect.")
            break

        # Pause to avoid rate limits after each iteration
        time.sleep(2)

    except Exception as e:
        # Handle rate limits or unexpected errors
        print(f"An error occurred: {e}")
        print("Sleeping for 60 seconds before retrying...")
        time.sleep(60)

# Create DataFrame from the collected data
posts_df = pd.DataFrame(posts_data)

print(posts_df.shape)

# Export the DataFrame to a CSV file
posts_df.to_csv('AskReddit_posts.csv', index=False)

print("Posts data exported successfully")


(5000, 9)
Posts data exported successfully
